In [63]:
import fairseq

In [69]:
from fairseq.models.roberta import RobertaModel

roberta = RobertaModel.from_pretrained('/projects/deepgreen/pstjohn/roberta_base_checkpoint/',
                                       checkpoint_file='checkpoint_best.pt')

state = fairseq.checkpoint_utils.load_checkpoint_to_cpu('/projects/deepgreen/pstjohn/roberta_base_checkpoint/checkpoint_best.pt')

In [77]:
state.keys()

dict_keys(['cfg', 'args', 'model', 'optimizer_history', 'extra_state', 'last_optimizer_state'])

In [70]:
import numpy as np
import scipy.sparse

In [71]:
train_annotations = scipy.sparse.load_npz('/projects/deepgreen/pstjohn/swissprot_go_annotation/fairseq_swissprot/label/train.npz')

total = np.prod(train_annotations.shape)
pos = train_annotations.sum()
neg = total - pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

initial_bias = np.array(np.log(train_annotations.sum(0) / train_annotations.shape[0] + np.finfo(float).eps)).squeeze()
initial_bias

In [72]:
roberta.model.register_classification_head('go_prediction', num_classes=initial_bias.shape[0])

In [73]:
bias = roberta.model.classification_heads['go_prediction'].out_proj.bias.data
roberta.model.classification_heads['go_prediction'].out_proj.bias.data = bias.new(initial_bias)

In [78]:
new_state = dict(state)
new_state['model'] = roberta.model.state_dict()

In [79]:
torch.save(new_state, '/projects/deepgreen/pstjohn/roberta_base_checkpoint/roberta.base_with_go_bias.pt')

In [80]:
roberta = RobertaModel.from_pretrained('/projects/deepgreen/pstjohn/roberta_base_checkpoint/',
                                       checkpoint_file='roberta.base_with_go_bias.pt')

In [81]:
roberta.model.classification_heads['go_prediction'].out_proj.bias.data

tensor([ -9.1192,  -8.1060,  -7.9205,  ..., -13.1623, -13.1623, -13.1623])